In [1]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
with open("yoga_poses_model.pkl", 'rb') as f:
    model = pickle.load(f)

In [27]:
df = pd.read_csv('dataset.csv')

In [29]:
df

,class,x0,y0,z0,v0,x1,y1,z1,v1,x2,...,z30,v30,x31,y31,z31,v31,x32,y32,z32,v32
0,Adho mukha svanasana - Downward-facing Dog,0.490031,0.759308,-0.057471,0.999915,0.500056,0.755584,-0.047095,0.999920,0.500699,...,0.052691,0.996797,0.279771,0.833945,0.243554,0.910503,0.270018,0.856671,-0.025365,0.998438
1,Adho mukha svanasana - Downward-facing Dog,0.485790,0.760255,-0.029428,0.999592,0.496677,0.760252,-0.017626,0.999563,0.497819,...,0.055273,0.995301,0.287544,0.827749,0.228765,0.854848,0.276033,0.847152,-0.026866,0.997748
2,Adho mukha svanasana - Downward-facing Dog,0.481833,0.749507,-0.061982,0.999264,0.493204,0.754042,-0.052008,0.999340,0.494498,...,-0.018221,0.998571,0.309814,0.819004,0.178746,0.939306,0.284677,0.841712,-0.099234,0.999354
3,Adho mukha svanasana - Downward-facing Dog,0.491385,0.745805,-0.039555,0.999747,0.501969,0.748230,-0.028987,0.999742,0.503439,...,0.025266,0.997481,0.321623,0.834647,0.203519,0.905652,0.311496,0.856482,-0.055656,0.998913
4,Adho mukha svanasana - Downward-facing Dog,0.468007,0.755088,-0.107148,0.999898,0.479416,0.760685,-0.094679,0.999896,0.480828,...,0.062055,0.997080,0.322556,0.829728,0.251807,0.924806,0.307081,0.849876,-0.026802,0.998917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,Uttanasana - Standing Forward Bend,0.563940,0.709303,-0.015861,0.999877,0.572540,0.718796,-0.006796,0.999551,0.574079,...,0.091428,0.925738,0.539025,0.864815,0.226189,0.443449,0.541203,0.869124,0.025284,0.947647
509,Uttanasana - Standing Forward Bend,0.579806,0.715018,0.004251,0.999988,0.592302,0.714799,0.015915,0.999966,0.593242,...,0.043072,0.955579,0.540875,0.861356,0.192298,0.617509,0.542630,0.872515,-0.026698,0.981065
510,Uttanasana - Standing Forward Bend,0.579068,0.711553,-0.043414,0.999859,0.590177,0.710548,-0.031884,0.999576,0.591256,...,0.051361,0.851145,0.542995,0.853005,0.161182,0.257018,0.543919,0.863454,-0.031308,0.904901
511,Uttanasana - Standing Forward Bend,0.577557,0.711253,-0.013271,0.999894,0.589083,0.709289,-0.002676,0.999828,0.589928,...,0.010511,0.923972,0.544783,0.851775,0.110742,0.431823,0.542060,0.862494,-0.068584,0.951600


In [48]:
cap = cv2.VideoCapture("/home/ms/BeCode/Projects/Yoga_class_analysis/BeCode x Yogalive - Pose recognition project/BeCode x Yogalive - Pose recognition project/test recording/testing_GMT20210504-174636_Recording_640x360.mp4")
#cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
          # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            #print(landmarks)
            
            # Get coordinates
            row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in landmarks]).flatten())
            #print(len(row))
            X = pd.DataFrame([row])
 
            model_class = model.predict(X)[0]
            print(model_class)
                 # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.putText(image, model_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            
        except:
            pass
        
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
3
3
2
2
3
3
3
3
3
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [21]:
image_test = [[0.5722866654396057, 0.7017782926559448, -1.059757947921753, 0.5881415009498596, 0.6075530052185059, -0.9736541509628296, 0.6056885719299316, 0.6051280498504639, -0.9739683866500854, 0.6211637854576111, 0.6023985147476196, -0.974108874797821, 0.5314260721206665, 0.6117583513259888, -0.9927152395248413, 0.5066487193107605, 0.612163782119751, -0.992692768573761, 0.4832099378108978, 0.6125773191452026, -0.992660403251648, 0.6331517696380615, 0.6344770193099976, -0.4833148121833801, 0.4523887634277344, 0.6503410339355469, -0.5530497431755066, 0.6008699536323547, 0.7791348099708557, -0.8733795285224915, 0.5324116349220276, 0.7920336127281189, -0.894945502281189, 0.7410592436790466, 0.9975823163986206, -0.23390308022499084, 0.3210917115211487, 1.0356996059417725, -0.3447815477848053, 0.8433715105056763, 1.4039710760116577, -0.6559368968009949, 0.25945302844047546, 1.5544986724853516, -0.5632027387619019, 0.739930272102356, 1.0949623584747314, -1.4981497526168823, 0.30888867378234863, 1.7583892345428467, -1.1571688652038574, 0.7482938766479492, 1.1816236972808838, -1.6656166315078735, 0.30411869287490845, 1.8433395624160767, -1.2922682762145996, 0.7024887800216675, 0.9455780982971191, -1.6350300312042236, 0.33614128828048706, 1.7964097261428833, -1.2995574474334717, 0.7038765549659729, 0.9908173680305481, -1.5166140794754028, 0.349570631980896, 1.7661973237991333, -1.1785991191864014, 0.673978328704834, 2.003342390060425, 0.035555411130189896, 0.40859755873680115, 2.0252294540405273, -0.024743080139160156, 0.7074674963951111, 2.7538747787475586, -0.5422829389572144, 0.4506458640098572, 2.80329966545105, -0.31030377745628357, 0.7083703279495239, 3.45116925239563, 0.05492708086967468, 0.46953850984573364, 3.4844143390655518, 0.3918929100036621, 0.7051359415054321, 3.5568768978118896, 0.09012877196073532, 0.4657401442527771, 3.5868988037109375, 0.43225011229515076, 0.7021046280860901, 3.6689367294311523, -0.4353659749031067, 0.5288488268852234, 3.695413589477539, -0.09303481131792068]]

In [16]:
import pandas as pd
image_test = np.array(image_test)


In [18]:
image_test = image_test.reshape(-1,1)

In [22]:
predict = model.predict(image_test)

ValueError: X.shape[1] = 99 should be equal to 132, the number of features at training time